In [ ]:
import pandas as pd
import numpy as np
#import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load data
news = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Final/archive/all_news.csv')

In [ ]:
news.head()

,title,content,type
0,"""Jihad"" Jack Thomas acquitted of terrorism con...",Melbourne man Jack Thomas has been acquitted o...,political
1,'Afghanistan is a 20-year venture' warns Canad...,"Maj.-Gen. Andrew Leslie, one of Canada's top g...",political
2,2006 New Zealand Maori Electoral Option,Today marks the beginning of the 2006 Māori El...,political
3,2008 GOP convention to be in Minnesota,The Republican National Committee Site Selecti...,political
4,30 civillians killed by roadside bomb in Afgha...,Local officials in southern Afghanistan say a ...,political


In [ ]:
news = news[~((news['type'] == 'unknown')|(news.type == 'rumor'))]

In [ ]:
news.groupby('type').size().sort_values(ascending = False)

type
political     265791
unreliable    139659
bias          124074
fake          121772
conspiracy     76658
clickbait      20917
junksci        15029
satire         14252
reliable        5417
hate            3567
dtype: int64

In [ ]:
# marge labels
news['type'].replace(['political','reliable','clickbait'], 1, inplace = True)
news['type'].replace(['fake','bias','conspiracy','junksci','unreliable','hate','satire'], 0, inplace = True)
news['type']=pd.to_numeric(news['type'])
# remove NaN
#news = news.dropna()

In [ ]:
news.groupby('type').size().sort_values(ascending = False)

type
0    495011
1    292125
dtype: int64

In [ ]:
sub_news=news.sample(frac=0.1,random_state=42)

In [ ]:
sub_news=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Final/processed_sub_news.csv')

In [ ]:
# without title
wot_sub_news= sub_news[['content', 'type']]
wot_sub_news= wot_sub_news.dropna()
# # with title
# wt_sub_news = sub_news[['title','content', 'type']]
# wt_sub_news= wt_sub_news.dropna()

In [ ]:
liar = pd.read_csv('/content/test.csv',header=None)

In [ ]:
liar.groupby(1).size().sort_values(ascending = False)

1
half-true      265
FALSE          249
mostly-true    241
barely-true    212
TRUE           208
pants-fire      92
dtype: int64

In [ ]:
liar = liar[((liar[1] == 'FALSE')|(liar[1] == 'TRUE'))]

In [ ]:
liar[1].replace(['FALSE'], 0, inplace = True)
liar[1].replace(['TRUE'], 1, inplace = True)

In [ ]:
liar.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,11972.json,1,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview
1,11685.json,0,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference
2,11096.json,0,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.
5,5962.json,1,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",brendan-doherty,NaN,Rhode Island,republican,1,2,1,1,0,a campaign website
6,7070.json,1,Says that Tennessee law requires that schools ...,"county-budget,county-government,education,taxes",stand-children-tennessee,Child and education advocacy organization.,Tennessee,none,0,0,0,0,0,in a post on Facebook.


In [ ]:
kaggle = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Final/kaggle.csv')

In [ ]:
kaggle=kaggle[((kaggle['label'] == 'REAL')|(kaggle['label'] == 'FAKE'))]

In [ ]:
kaggle['label'].replace(['FAKE'], 0, inplace = True)
kaggle['label'].replace(['REAL'], 1, inplace = True)

# tf-idf

In [ ]:
# Import Tfidf vectorizer from sklearn and apply the vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(max_features = 256)
content_tfidf = vect.fit_transform(wot_sub_news.content)

In [ ]:
content_wt_tfidf = vect.fit_transform(wt_sub_news['content']+wt_sub_news['title'])

In [ ]:
liar_tfidf = vect.fit_transform(liar[2])

In [ ]:
kaggle_tfidf = vect.fit_transform(kaggle['text'])

# Training a classifier

### Split the data into training, validation and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
# Size of training is 80%
X_train, X_test, y_train, y_test = train_test_split(
    content_wt_tfidf, wt_sub_news['type'], test_size=0.20, random_state=42)

# Use the 40% test set to split further into test and validation set with 50/50 split
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

names = ["Nearest Neighbors","Random Forest", "Neural Net","LogReg"]#"SVC"]

classifiers = [
    KNeighborsClassifier(3),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    DecisionTreeClassifier(max_depth=5),
    LogisticRegression(penalty='l1',solver='saga', max_iter=10000)]
    #SVC(kernel='linear')]

for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        print(name,' score:',clf.score(X_test, y_test))
        print(name,' liar score:',clf.score(liar_tfidf, liar[1]))

Nearest Neighbors  score: 0.7675098512774883
Nearest Neighbors  liar score: 0.5361050328227571
Random Forest  score: 0.6710308885216728
Random Forest  liar score: 0.5448577680525164
Neural Net  score: 0.7503495614592602
Neural Net  liar score: 0.5361050328227571
LogReg  score: 0.8107283589678403
LogReg  liar score: 0.474835886214442


In [ ]:
from sklearn.model_selection import train_test_split
# without title
# Size of training is 60%
X_train, X_test, y_train, y_test = train_test_split(
    content_tfidf, wot_sub_news['type'], test_size=0.20, random_state=42)

# Use the 40% test set to split further into test and validation set with 50/50 split
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC
names = ["Nearest Neighbors","Random Forest", "Neural Net","LogReg"]#,"SVC"]

classifiers = [
    KNeighborsClassifier(3),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    DecisionTreeClassifier(max_depth=5),
    LogisticRegression(penalty='l1',solver='saga', max_iter=10000)]
    #SVC(kernel='linear')]

for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        #print(name,' score:',clf.score(X_test, y_test))
        #print(name,' liar score:',clf.score(liar_tfidf, liar[1]))
        #print(name,' kaggle score:',clf.score(kaggle_tfidf, kaggle['label']))


In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.87      0.85      5387
           1       0.75      0.69      0.72      2948

    accuracy                           0.81      8335
   macro avg       0.79      0.78      0.79      8335
weighted avg       0.81      0.81      0.81      8335



In [ ]:
kaggle_challenge=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Final/kaggle_json_to_csv.csv')

In [ ]:
kaggle_challenge['article'] =kaggle_challenge['article'].astype(str)

In [ ]:
kaggle_challenge_tfidf = vect.fit_transform(kaggle_challenge['article'])

In [ ]:
kaggle_challenge['label'] = clf.predict(kaggle_challenge_tfidf)

In [ ]:
kaggle_challenge['label'] = np.where(kaggle_challenge['label']==0, 'FAKE', 'REAL')

In [ ]:
res = kaggle_challenge[['id','label']]
res.to_csv('res.csv',index=False)